## 707 topics classification using transformer based sentence embedding model "all-mpnet-base-v2" and custom NN architecture

Danilchenko Vadim

In [1]:
import pandas as pd
import numpy as np
import joblib, re, string
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Dense, Input, Dropout, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [2]:
from sentence_transformers import SentenceTransformer
vectorizer = SentenceTransformer('model/all-mpnet-base-v2')

In [3]:
# удалим приветствия
def del_intro(text):

        alphanum = r'a-zа-яёе0-9'
        alphabet = r'a-zа-яёе'

        del_from_sentence = ['добрый день', 'добрый вечер', 'доброе утро', 'просьба подсказать', 'доброй ночи', 'доброе время суток', 'доброго времени суток', 'доброго времени', \
                             'день добрый', 'доброго времени', 'доброго дня', 'вечер добрый', 'утро доброе', 'подскажите пожалуйста', 'подскажите, пожалуйста', 'подскажите,пожалуйста', \
                             'скажите пожалуйста', 'скажите, пожалуйста', 'скажите,пожалуйста']

        add2stop_words = ['подскажите', 'подсказать', 'пожалуйста', 'здравствуйте', 'здраствуйте', 'здравствуй', 'пожалуйста', 'приветствую', 'привет']

        stop_words = del_from_sentence + add2stop_words

        text = str(text).lower()
        for word in stop_words:
                if word in text:
                        # print(word)
                        text = re.sub(word, ' ', text)
        obj = re.match(r'[\d+\W+]{,}', text)
        text = text[0 if obj is None else len(obj.group(0)):]
        text = re.sub(r'[\s+]', ' ', text)
        text = re.sub(r'([a-zа-яёе])\1{2,}', r'\1\1', text)  # aaaaa -> aa
        text = re.sub(r'([^a-zа-яёе0-9])\1{1,}', r'\1', text)  # )))) -> )
        text = re.sub(r'([^0-9]|^)([%s]+)([^0-9]|$)' % string.punctuation, r'\1 \2 \3', text)
        return text

In [5]:
# загрузим датасет
df = pd.read_excel('data/train_data.xlsx')
df.shape

(138695, 14)

In [7]:
df.columns[:6]

Index(['intent_id', 'domain', 'labeled', 'message', 'mode', 'intent'], dtype='object')

In [8]:
# сгруппируем по тематике и посчитаем кол-во примеров
df_grouped = df.groupby('intent').agg({'intent_id':'count'}).reset_index()
df_grouped.columns=['intent', 'cnt']#.sort_values(ascending=True)
df_grouped.sort_values(by='cnt', ascending=False, inplace=True)
print('№ intents:', df_grouped.shape[0])

№ intents: 707


In [9]:
# посмотрим на распределение кол-ва примеров на тематиках
df_grouped['cnt'].describe()

count     707.000000
mean      196.173975
std       348.631691
min         1.000000
25%        34.000000
50%        65.000000
75%       196.500000
max      3600.000000
Name: cnt, dtype: float64

In [10]:
# выведем кол-во примеров на тематику до 10го процентиля по выборке
np.percentile(df_grouped['cnt'].tolist(), 10)

15.0

In [11]:
# выведем кол-во тематик, в которых кол-во примеров ниже 10го процентиля по выборке
df_grouped[df_grouped['cnt']<=np.percentile(df_grouped['cnt'].tolist(), 10)].shape

(72, 2)

10% от выборки - тематики, с количеством примеров менее 15. 
Тк в данном примере не предусматривается апсемплинг, имеем ввиду, что точность на данных тематиках опустит общий скор вниз

In [12]:
# выполним препроцессинг
df['preprocessed'] = df['message'].apply(del_intro)

---

### векторизуем с помощью Sentence Embedding модели на архитектуре трансформеров

In [14]:
%%time
vectors = vectorizer.encode(df['preprocessed'].tolist())
vectors.shape

CPU times: user 8min 53s, sys: 9min 21s, total: 18min 15s
Wall time: 8min


----------

In [15]:
# подготовим классы
df['target'] = df['intent'].astype('category').cat.codes
df['target'].head()

0    13
1    13
2    13
3    14
4    14
Name: target, dtype: int16

In [16]:
# создадим словарь отношений номера класса к названию тематик
intent2target = {}
for i, row in df[['intent', 'target']].drop_duplicates().iterrows():
    intent2target[row['target']] = row['intent']

len(intent2target)

707

In [17]:
# преобразуем в категориальный вид таргет
y_cat = to_categorical(df['target'].tolist())

Using TensorFlow backend.


In [18]:
# разделим на тренировочную и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(vectors, y_cat, test_size=0.1, random_state=777, shuffle=True)
print('shapes: x_train, x_test, y_train, y_test:', x_train.shape, x_test.shape, y_train.shape, y_test.shape)

shapes: x_train, x_test, y_train, y_test: (124825, 768) (13870, 768) (124825, 707) (13870, 707)


-----------

### обучим модель нейросети на эмбеддингах модели all-mpnet-base-v2

In [31]:
x_train[0][:20]

array([-0.00681733, -0.04985548,  0.04259589, -0.01603217,  0.04686847,
       -0.00667169, -0.01182848,  0.01598987, -0.00153588,  0.00027185,
        0.02500646, -0.02011023,  0.00300912,  0.02421119,  0.03337623,
       -0.03482555, -0.02754534,  0.01827239, -0.01404077,  0.03275978],
      dtype=float32)

In [32]:
inp_ = Input(shape=(x_train.shape[1],))
x = Dense(512, activation='linear', name='dense1')(inp_)
x = Dropout(0.2)(x)
x = Dense(512, activation='linear', name='dense2')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='linear', name='dense3')(x)
x = Dropout(0.3)(x)
out = Dense(y_train.shape[1], activation='softmax', name='dense_out')(x)
model = Model(inp_, out)

model.compile(optimizer=Adam(learning_rate=0.0001),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# model.load_weights('model/weights/model_weights_0.001.hdf5')

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 768)               0         
_________________________________________________________________
dense1 (Dense)               (None, 512)               393728    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense2 (Dense)               (None, 512)               262656    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense3 (Dense)               (None, 512)               262656    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0   

In [33]:
r = model.fit(x_train,
             y_train,
             epochs=25,
             batch_size=2048,
             validation_data=(x_test, y_test),
             verbose=1,
             callbacks=[ModelCheckpoint('model/weights/model_weights_0.0001.hdf5',
                                    monitor='val_accuracy',
                                    mode='max',
                                    save_weights_only=True,
                                    save_best_only=True,
                                    verbose=0)])

Train on 124825 samples, validate on 13870 samples
Epoch 1/25
124825/124825 [==============================] - 4s 31us/step - loss: 0.5540 - accuracy: 0.8322 - val_loss: 0.8051 - val_accuracy: 0.7973
Epoch 2/25
124825/124825 [==============================] - 3s 25us/step - loss: 0.5443 - accuracy: 0.8337 - val_loss: 0.8035 - val_accuracy: 0.7973
Epoch 3/25
124825/124825 [==============================] - 3s 23us/step - loss: 0.5393 - accuracy: 0.8340 - val_loss: 0.8019 - val_accuracy: 0.7985
Epoch 4/25
124825/124825 [==============================] - 3s 26us/step - loss: 0.5398 - accuracy: 0.8356 - val_loss: 0.7987 - val_accuracy: 0.7980
Epoch 5/25
124825/124825 [==============================] - 3s 25us/step - loss: 0.5391 - accuracy: 0.8354 - val_loss: 0.7990 - val_accuracy: 0.7987
Epoch 6/25
124825/124825 [==============================] - 3s 24us/step - loss: 0.5372 - accuracy: 0.8359 - val_loss: 0.7975 - val_accuracy: 0.7987
Epoch 7/25
124825/124825 [=============================

посмотрим на результат дообучения модели на разных lr

In [22]:
model.load_weights('model/weights/model_weights_0.01.hdf5')
model.evaluate(x_test, y_test)

13870/13870 [==============================] - 1s 93us/step


[1.1857967839278845, 0.7187454700469971]

In [25]:
model.load_weights('model/weights/model_weights_0.001.hdf5')
model.evaluate(x_test, y_test)

13870/13870 [==============================] - 1s 95us/step


[0.8256376277394835, 0.7938716411590576]

In [34]:
model.load_weights('model/weights/model_weights_0.0001.hdf5')
model.evaluate(x_test, y_test)

13870/13870 [==============================] - 1s 97us/step


[0.7985499518847586, 0.8011535406112671]

------------